# Get PDF Files for publications in the UK Catalysis Hub app db
A list of publications is obtainded from the app database. This list will contain a titles, IDs and DOIs which need to be explored to look for the corresponding pdf files. 
The steps of the process are: 
 1. get a Title, DOI, and URL for each publication
 2. convert the DOI to a pdf file name and try to open de file
 3. use pdfMiner and/or CDE to get the reference to data
 4. add a new dataset entry each time a new data object is found

In [1]:
# Libraries
# library containign functions that read and write to csv files
import lib.handle_csv as csvh
# library for connecting to the db
import lib.handle_db as dbh
# library for handling text matchings
import lib.text_comp as txtc
# library for getting data from crossref
import lib.crossref_api as cr_api
# library for handling url searchs
import lib.handle_urls as urlh
# managing files and file paths
from pathlib import Path
# add aprogress bar
from tqdm import tqdm_notebook 
#library for handling json files
import json
# library for using regular expressions
import re
# library for handling http requests
import requests

# import custom functions (common to various notebooks)
import processing_functions as pr_fns



In [2]:
# functions for ChemDataExtractor
# not used for mining data references (suplementary/raw) or to get pdf metadata
from chemdataextractor import Document

# A function for getting a list of files from the directory
# This will be modified to get the list from a csv file
def get_files_list (source_dir):
    i_counter = 0
    files_list = []
    for filepath in sorted(source_dir.glob('*.pdf')):
        i_counter += 1
        files_list.append(filepath)
    return files_list

def cde_read_pdfs(a_file):
    pdf_f = open(a_file, 'rb')
    doc = Document.from_file(pdf_f)
    return doc

def find_doi(element_text):
    cr_re_01 = '10.\d{4,9}/[-._;()/:A-Z0-9]+'
    compare = re.search(cr_re_01, element_text, re.IGNORECASE)
    if compare != None:
        return compare.group()
    return ""

def get_db_id(doi_value, db_name = "app_db.sqlite3"):
    db_conn = dbh.DataBaseAdapter(db_name)
    table = 'articles'   
    id_val = db_conn.get_value(table, "id", "doi", doi_value)
    db_conn.close()
    if id_val != None:
        return id_val[0]
    else:
        return 0

def get_db_title(doi_value, db_name = "app_db.sqlite3"):
    db_conn = dbh.DataBaseAdapter(db_name)
    table = 'articles'   
    id_val = db_conn.get_value(table, "title", "doi", doi_value)
    db_conn.close()
    if id_val != None:
        return id_val[0]
    else:
        return 0

def get_close_dois(str_name, db_name = "prev_search.sqlite3"):
    db_conn = dbh.DataBaseAdapter(db_name)
    search_in = 'articles'
    fields_required = "id, doi, title, pdf_file"
    filter_str = "doi like '%"+str_name+"%';"

    db_titles = db_conn.get_values(search_in, fields_required, filter_str)
    db_conn.close()
    return db_titles

Get the name of the current app db file:

In [ ]:
# app db file with path: db_files/app_db.sqlite3
ukchapp_db = "db_files/app_db.sqlite3"
while not Path(ukchapp_db).is_file():
    print('Please enter the name of app db file:')
    ukchapp_db = input()
    


## Get pdf files for publications

Read database and try to recover pdf files

In [ ]:
# get publication data from the ukch app
db_pubs = pr_fns.get_pub_app_data(ukchapp_db)

# get the list of dois already mined for data 
input_file = 'pub_data_all.csv'
id_field = 'num'
processed, headings = csvh.get_csv_data(input_file, id_field)
for id_num in processed:
    current_title = processed[id_num]['doi']
processed[1]['num']

processed_dois = []
for entry in processed:
    if not processed[entry]['doi'] in processed_dois:
        processed_dois.append( processed[entry]['doi'])


for a_pub in tqdm_notebook(db_pubs):
    pub_id = a_pub[0]
    pub_title = a_pub[1]
    pub_doi = a_pub[2]
    pub_url = a_pub[3]
    pub_pdf = a_pub[4]
    pub_html = a_pub[5]
    if pub_pdf == 'None':
        print("*************************")
        print("Missing PDF for:", pub_doi)
        print("*************************")
    else:
        pdf_file = "pdf_files/" + pub_pdf
        if not Path(pdf_file).is_file():
            print("*************************")
            print("Missing file for:", pdf_file, "for", pub_doi)
            print("*************************")
        else: 
            print("PDF filename", pub_pdf)
            

## Use pdfminer to get metadata from pdf file

In [3]:
pdf_file = "pdf_files/" + 's41929-019-0334-3.pdf'
import pdfminer
from pdfminer.high_level import extract_text
text = extract_text(pdf_file)

In [56]:
def get_ref_sentences(text):
    sentences = text.split("\n")
    groups=[]
    for sentence in sentences:
        if pr_fns.is_data_stmt(sentence.lower()):
            idx = sentences.index(sentence)
            #context = sentences[idx-1].strip() + sentences[idx].strip() + sentences[idx+1].strip()
            groups.append([idx-1,idx,idx+1])
           # print(sentences.index(sentence), ":" , len(sentence), sentence )
            #print(context)

    #print(groups)

    reduced_groups = []
    for group in groups:
        idx_group = groups.index(group)
        if groups.index(group) > 0:
            set_g = set(group)
            # make the array before current a set
            set_bg = set(groups[idx_group - 1])
            # make the array after current a set
            set_ag = set()
            if idx_group + 1 < len(groups):    
                set_ag = set(groups[idx_group + 1])
            if len(set_bg.intersection(set_g)) > 0:
                #print(set_bg.intersection(set_g), set_bg, set_g, set_bg.union(set_g))
                ordered_union = list(set_bg.union(set_g))
                ordered_union.sort()
                #print(ordered_union)
                reduced_groups.append(ordered_union)
            if len(set_ag.intersection(set_g)) > 0:
                #print(set_ag.intersection(set_g), set_ag, set_g)
                ordered_union = list(set_ag.union(set_g))
                ordered_union.sort()
                reduced_groups.append(ordered_union)
            if len(reduced_groups) > 0:
                is_in_rg = False
                for a_rg in reduced_groups:
                    if set_g.issubset(a_rg):
                        is_in_rg = True
                        break
                if not is_in_rg:
                    reduced_groups.append(list(set_g))
            #print(reduced_groups)

    return_group = []
    for sentence_group in reduced_groups:
        full_sentence = ""
        for single_sentence in sentence_group:
            full_sentence += sentences[single_sentence].strip()
        return_group.append(full_sentence)

    return return_group


def get_http_ref(sentence):
    http_frag = ""
    if 'http' in sentence.lower():
        idx_http = sentence.lower().index('http')
        http_frag = sentence[idx_http:]
        space_in_ref = True
        while " " in http_frag:
            space_idx = http_frag.rfind(" ")
            http_frag = http_frag[:space_idx]
        if(http_frag[-1:]=="."):
            http_frag = http_frag[:-1]
    return http_frag
    

print(get_ref_sentences(text))
ref_sentences = get_ref_sentences(text)
for a_sentence in ref_sentences:
    print(get_http_ref(a_sentence))

#for sentence in sentences:
#    if 'data' in sentence.lower():
#        print (len(sentence), sentence, sentences.index(sentence))


['ArticlesNature Catalysis\x0cData availabilityInformation on the data supporting the results presented here, including how toaccess them, can be found in the Cardiff University data catalogue at https://doi.org/10.17035/d.2019.0079744472.', 'Additional informationSupplementary information is available for this paper at https://doi.org/10.1038/s41929-019-0334-3.']
https://doi.org/10.17035/d.2019.0079744472
https://doi.org/10.1038/s41929-019-0334-3


In [ ]:
for sentence in sentences:
    print(sentences.index(sentence), ":" , len(sentence), sentence ) 

In [51]:
sentence = "can be found in the Cardiff University data catalogue at https://doi.org/10.17035/d.2019.0079744472. At C 1"
pr_fns.is_data_stmt(sentence.lower())

support_keys = ["data", "underpin", "support", "result", "found", "find", "obtain", "doi","raw", 'information',
                    "provide", "available", "online", "supplement"]
print (sentence.lower())
count = 0
for a_word in support_keys:
    if a_word in sentence.lower():
        print(a_word)
        count += 1
print (count)

if 'http' in sentence.lower():
    idx_http = sentence.lower().index('http')
    http_frag = sentence[idx_http:]
    space_in_ref = True
    while " " in http_frag:
        space_idx = http_frag.rfind(" ")
        http_frag = http_frag[:space_idx]
        print(http_frag.rfind(" "))
    if(http_frag[-1:]=="."):
        http_frag = http_frag[:-1]
    print(http_frag)



can be found in the cardiff university data catalogue at https://doi.org/10.17035/d.2019.0079744472. at c 1
data
found
doi
3
46
43
-1
https://doi.org/10.17035/d.2019.0079744472


## File name match

In [ ]:
# check if file name matches some part of a doi
files_list = get_not_matched_files(ukchapp_db)

not_assigned = []
for a_file in tqdm_notebook(files_list):
    search_this = a_file.name.replace(".pdf", "").lower()
    print(a_file.name,"\t",search_this)
    close_dois = get_close_dois(search_this, ukchapp_db)
    print(len(close_dois))
    
    if len(close_dois) == 1 :
        doi_dat = close_dois[0]
        selected = False
        if doi_dat[3] == None:
            while not selected:
                print("Assign file: ", a_file.name, " to:\n\t", doi_dat[0],doi_dat[1],doi_dat[2], doi_dat[3])
                print('***************************************************************')
                print("Options:\n\ta) assign\n\tb)go to next")
                print("selection:")
                usr_select = input()
                if usr_select == 'a':
                    selected = True
                    set_pdf_file_value(a_file.name, doi_dat[0], ukchapp_db)
                    print("assing and go to next")
                elif usr_select == 'b':
                    #working_file[art_num]['ignore']=3 # visual inspection
                    selected = True
                    print("going to next")
        else:
            print("Assigned in db: ",  doi_dat[0],doi_dat[1],doi_dat[2], doi_dat[3])
    else:
        not_assigned.append(a_file)        

In [ ]:
files_list = get_files_list(Path("pdf_files"))
db_pubs = get_pub_app_data(ukchapp_db)
missing=[]
# check which files are really missing linking
for file in files_list:
    found_in_db = False
    for db_pub in db_pubs:
        if file.name == db_pub[4]:
            found_in_db = True
            break
    if not found_in_db:
        missing.append(file)

# check if all linked files are in the folder
missing2=[]
for db_pub in db_pubs:
    found_in_system = False
    for file in files_list:
        if file.name == db_pub[4] or db_pub[4] == None:
            found_in_system = True
            break
    if not found_in_system:
        missing2.append(db_pub)


In [ ]:
# use ChemDataExtractor to read pdf and get DOIs in document
for a_file in tqdm_notebook(not_assigned):
    pdf_doc = cde_read_pdfs(a_file)
    print(a_file.name)
    dois_list = []
    for element in pdf_doc.elements:
        if 'doi' in str(element):
            found_doi = find_doi(str(element))
            if found_doi[-1:] == ".":
                found_doi = found_doi[:-1]
            if not found_doi in dois_list:
                dois_list.append(found_doi)       
    
    if dois_list != [] and len(dois_list) == 1:
        for a_doi in dois_list:
            close_dois = get_close_dois(a_doi, ukchapp_db)
            selected = False
            if len(close_dois) == 1:
                doi_dat = close_dois[0]
                if doi_dat[3] == None:
                    while not selected:
                        print("Assign file: ",a_file.name, " to:\n\t", doi_dat[0],doi_dat[1],doi_dat[2], doi_dat[3])
                        print('***************************************************************')
                        print("Options:\n\ta) assign\n\tb)go to next")
                        print("selection:")
                        usr_select = input()
                        if usr_select == 'a':
                            selected = True
                            set_pdf_file_value(a_file.name, doi_dat[0], ukchapp_db)
                            print("assing and go to next")
                        elif usr_select == 'b':
                            #working_file[art_num]['ignore']=3 # visual inspection
                            selected = True
                            print("going to next")
                else: 
                    print("Already assingned to:\n\t", doi_dat[0],doi_dat[1],doi_dat[2], doi_dat[3])
                